2023/02/16<br>

This ocde is for getting retardation, azimuthal angle and MFA images based on PolSope algorithm.<br>
After conversion, all images are concatenated by Phase-only-orrelated method.<br>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import itertools
import os
import gc
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool
from module.im_stitching import *
from module.Tool_box import *
import copy

In [2]:
current_path=os.getcwd()
image_path=os.path.join(current_path, "FLM")
species=os.listdir(image_path)
npz_path=os.path.join(current_path, "npz_file")
save_path=os.path.join(current_path, "fig_save")

path_list=[[npz_path, "FLM_temp_save"], [save_path, "FLM_temp"]]

for path in path_list:
    create_folder(path[0], path[1])

#find samlpe name from dierctory
sample_path_list=[]
for curDir, dirs, files in os.walk(image_path):
    if len(files)>1:
        sample_path_list.append(curDir)

In [3]:
#adjust image size
FLM_x20=108.237 #nm/pixel
Polscope_x40=160.256 #nm/pixel

scaling_factor=FLM_x20/Polscope_x40

In [4]:
file_path_list=[]
sample_position_list=[]

for sample_path in tqdm(sample_path_list):
    #find target files
    file_path_list_temp=find_target(sample_path, extension="tif")
    file_path_list_temp=list(itertools.chain.from_iterable(file_path_list_temp))
    
    #get target position
    sample_position_list_temp=[os.path.splitext(os.path.basename(file_path_list_temp[i]))[0].split("_")[-2:]
                                   for i in range((len(file_path_list_temp)))]
    
    #create batches of image paths
    height_div=int(os.path.splitext(file_path_list_temp[-1])[0].split("_")[-2])+1
    r_num=len(file_path_list_temp)//height_div
    file_path_rearrange=[file_path_list_temp[r_num*i:r_num*(i+1)] for i in range(len(file_path_list_temp)//r_num)]
    
    #save results
    file_path_list.append(file_path_rearrange)
    sample_position_list.append(sample_position_list_temp)

100%|██████████| 11/11 [00:00<00:00, 3971.20it/s]


**Stitch images and save them as npz**<br>
This step is done by multisteps for considering memory handling.

In [8]:
for i, file_path_sp in enumerate(file_path_list[9:10]):
    for file_path_batch in tqdm(file_path_sp):
        #get row number
        row_No=os.path.splitext(file_path_batch[0])[0].split("_")[-2]
        
        #load image
        FLM=list(map(load_im, file_path_batch))
        FLM=[resize_im(FLM[j], scaling=scaling_factor, interpolation=cv2.INTER_NEAREST) 
             for j in range(len(FLM))]
        
        """
        FLM_norm=[]
        for j in range(len(FLM)):
            target_FLM=FLM[j]
            target_layer=target_FLM[:, :, 2]
            target_layer=(target_layer-np.min(target_layer))/(np.max(target_layer)-np.min(target_layer))*255
            target_FLM[:, :, 2]=target_layer
            FLM_norm.append(np.uint8(target_FLM))
        """
        
        ####Binarization####
        FLM_b=[binarize_im(FLM[j][:, :, 2], inv=True) for j in range(len(FLM))]

        #if mulitiple images exist in row direction, sticthing is performed.
        #else, original image is returned.
            
        
        ####image stitching####
        #if mulitiple images exist in row direction, sticthing is performed.
        #else, original image is returned.
        if len(FLM_b)>1:
            #image stitching in row directions
            FLM_b_stitch_row, FLM_stitch_row = stitch_im_POC(FLM_b, FLM, ref_padding=255, target_padding=0)

        else:
            FLM_b_stitch_row = FLM_b[0]
            FLM_stitch_row = FLM[0]
                
        #save the result of stitching
        sample_name=file_path_batch[0].split("/")[-2]
        
        create_folder(os.path.join(save_path, "FLM_temp"), str(sample_name))
        fig, ax=plt.subplots(1, 1)
        ax.imshow(FLM_stitch_row)
        ax.axis("off")
        plt.savefig(os.path.join(save_path, "FLM_temp", str(sample_name), 
                                                            str(sample_name)+"_rowNo_"+str(row_No)+".png"), dpi=450)
        plt.close()
        
        #create folder
        create_folder(os.path.join(npz_path, "FLM_temp_save"), str(sample_name))
        
        #save as npz
        np.savez_compressed(os.path.join(npz_path, "FLM_temp_save", str(sample_name), 
                                         str(sample_name)+"_rowNo_"+str(row_No)+".npz"), 
                            FLM=FLM_stitch_row, FLM_b=FLM_b_stitch_row)
        
        #delete list and release memory
        del FLM, FLM_b, FLM_b_stitch_row, FLM_stitch_row
        gc.collect()

100%|██████████| 3/3 [00:25<00:00,  8.58s/it]


**Reload stitching results and sticth them in column direction**

In [9]:
for i, sample_path in enumerate(sample_path_list[9:10]):
    sample_name=sample_path.split("/")[-1]
    target_path=os.path.join(npz_path, "FLM_temp_save", sample_name)
    npz_paths=find_target(target_path, extension=".npz")
    
    for npz_path_sp in tqdm(npz_paths):
        #load images
        FLM_list=[]
        FLM_b_list=[]
        for npz_path_ind in npz_path_sp:
            npz=np.load(npz_path_ind)
            FLM=npz["FLM"]

            #binarization
            FLM_b=binarize_im(FLM[:, :, 2], inv=False)

            #save results
            FLM_list.append(FLM)
            FLM_b_list.append(FLM_b)

        #image stitching in row directions
        FLM_b_stitch_column, FLM_stitch_column = stitch_im_POC(FLM_b_list, FLM_list)

        #save the result of stitching
        create_folder(os.path.join(save_path, "FLM_stitching_results"), str(sample_name))
        fig, ax=plt.subplots(1, 1)
        ax.imshow(FLM_stitch_column)
        ax.axis("off")
        plt.savefig(os.path.join(save_path, "FLM_stitching_results", str(sample_name), 
                                                            str(sample_name)+".png"), dpi=450)
        plt.close()

        #create folder
        create_folder(os.path.join(npz_path, "FLM_stitching_results"), str(sample_name))

        #save as npz
        np.savez_compressed(os.path.join(npz_path, "FLM_stitching_results", str(sample_name), 
                                         str(sample_name)+".npz"), 
                            FLM=FLM_stitch_column, FLM_b=FLM_b_stitch_column)

        #delete list and release memory
        del FLM_b_stitch_column, FLM_stitch_column, FLM_list, FLM_b_list
        gc.collect()

100%|██████████| 1/1 [00:15<00:00, 15.52s/it]
